# **Assignment 2:**
Predictive models


In [1]:
%load_ext autoreload
%autoreload 2

## The steps are:
1.   Load and explore dataset
2.   Data preparation
3.   Build Pipeline
4.   Split data
5.   Linear regression model
6.   Decision tree with tunning hyperparameters
7.   XGboost

## 1. Load and explore dataset

Import packages

In [2]:
import pandas as pd
import numpy as np

Load the dataset into dataframe

In [3]:
df_test = pd.read_csv('../../data/raw/sales_test.csv')
df_train = pd.read_csv('../../data/raw/sales_train.csv')
df_events = pd.read_csv('../../data/raw/calendar_events.csv')
df_calendar = pd.read_csv('../../data/raw/calendar.csv')
df_items_weekly_price = pd.read_csv('../../data/raw/items_weekly_sell_prices.csv')

Display the first 5 rows of tranining data 


In [4]:
df_train.head(5)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1532,d_1533,d_1534,d_1535,d_1536,d_1537,d_1538,d_1539,d_1540,d_1541
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,1,0,1,0,1,0,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,8,2,0,8,2,3,1,1,3,8
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,0,1,3,2,1,1,2,2,3


In [5]:
print("Missing values in tranining data:")
for i in df_train.columns[df_train.isna().any()]:
    no_missing = df_train[i].isna().sum()
    print("Column "+str(i) +":"+str(no_missing) +" NAs")

Missing values in tranining data:


Information of training dataset

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1547 entries, id to d_1541
dtypes: int64(1541), object(6)
memory usage: 359.9+ MB


Showing first 5 rows of calendar 

In [7]:
df_calendar.head(5)

,date,wm_yr_wk,d
0,2011-01-29,11101,d_1
1,2011-01-30,11101,d_2
2,2011-01-31,11101,d_3
3,2011-02-01,11101,d_4
4,2011-02-02,11101,d_5


Showing first 5 rows of event calendar 

In [8]:
df_events.head(5)

,date,event_name,event_type
0,2011-02-06,SuperBowl,Sporting
1,2011-02-14,ValentinesDay,Cultural
2,2011-02-21,PresidentsDay,National
3,2011-03-09,LentStart,Religious
4,2011-03-16,LentWeek2,Religious


Showing first 5 rows of test

In [9]:
df_test.head(5)

,d_1542,d_1543,d_1544,d_1545,d_1546,d_1547,d_1548,d_1549,d_1550,d_1551,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,0,1,0,2,1,0,2,0,1,0,...,2,4,0,0,0,0,3,3,0,1
1,0,0,0,0,0,0,0,0,1,0,...,0,1,2,1,1,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,4,1,0,1,3,5,2,3,0,2,...,1,1,0,4,0,1,3,0,2,6
4,3,0,0,1,1,0,2,0,2,1,...,0,0,0,2,1,0,0,2,1,0


## 2. Data preparations

Melt the training data

In [10]:
columnns=['id','dept_id','cat_id','state_id']
df_train_melt=pd.melt(df_train.drop(columns=columnns),id_vars=['item_id','store_id'],var_name='day',value_name='sales')

In [11]:
df_train_melt.head(5)

,item_id,store_id,day,sales
0,HOBBIES_1_001,CA_1,d_1,0
1,HOBBIES_1_002,CA_1,d_1,0
2,HOBBIES_1_003,CA_1,d_1,0
3,HOBBIES_1_004,CA_1,d_1,0
4,HOBBIES_1_005,CA_1,d_1,0


Combining calendar and events datasets

In [12]:
df_ce = pd.merge(df_calendar, df_events, on='date', how='left')

In [13]:
#drop the event name
df_ce.drop(columns="event_name",inplace=True)

In [14]:
#Rename day
df_ce.rename(columns={'d':'day'},inplace=True)

df_ce.head(5)

One date can have multiple events

In [15]:
print(df_ce[df_ce.duplicated('day')])

            date  wm_yr_wk     day event_type
86    2011-04-24     11113    d_86  Religious
829   2013-05-05     11315   d_828  Religious
1180  2014-04-20     11412  d_1178  Religious
1237  2014-06-15     11420  d_1234   Sporting
1973  2016-06-19     11621  d_1969   Sporting


In [16]:
df_ce[df_ce["day"]=="d_86"]

,date,wm_yr_wk,day,event_type
85,2011-04-24,11113,d_86,Cultural
86,2011-04-24,11113,d_86,Religious


Combine the malted dataset vs combined calendar and event.

In [17]:
df_train_ce = pd.merge(df_train_melt, df_ce, on='day', how='left')

Combine with items weekly price

In [18]:
df_train_combined = pd.merge(df_train_ce, df_items_weekly_price, on=['item_id','store_id','wm_yr_wk'], how='left')

In [19]:
df_train_combined.head(5)

,item_id,store_id,day,sales,date,wm_yr_wk,event_type,sell_price
0,HOBBIES_1_001,CA_1,d_1,0,2011-01-29,11101,NaN,NaN
1,HOBBIES_1_002,CA_1,d_1,0,2011-01-29,11101,NaN,NaN
2,HOBBIES_1_003,CA_1,d_1,0,2011-01-29,11101,NaN,NaN
3,HOBBIES_1_004,CA_1,d_1,0,2011-01-29,11101,NaN,NaN
4,HOBBIES_1_005,CA_1,d_1,0,2011-01-29,11101,NaN,NaN


Missing records:

In [20]:
print("Missing values in tranining data:")
for i in df_train_combined.columns[df_train_combined.isna().any()]:
    no_missing = df_train_combined[i].isna().sum()
    print("Column "+str(i) +":"+str(no_missing) +" NAs")

Missing values in tranining data:
Column event_type:43143350 NAs
Column sell_price:12291876 NAs


Fill null values in weekly sell price with average prices of the same item in different stores 

In [21]:
mean_price = df_train_combined.groupby(['item_id'])['sell_price'].transform('mean')

In [22]:
df_train_combined['sell_price'].fillna(mean_price, inplace=True)

Create a revanue column by using sales and sell_price

In [23]:
df_train_combined["revenue"]=df_train_combined["sales"]*df_train_combined["sell_price"]

Create a event columns to check if is there event on this day

In [24]:
df_train_combined['is_event'] = df_train_combined['event_type'].notnull().astype(int)

Drop sale, day, wm_yr_wk,sell_price, event_type columns

In [25]:
df_train_combined.drop(columns=["day","event_type","sales","wm_yr_wk","sell_price"],inplace=True)

Convert date to timestampe and then create new columns as day of month, month of year and day of week

In [26]:
df_train_combined['date'] = pd.to_datetime(df_train_combined['date'])
df_train_combined['day_of_month'] = df_train_combined['date'].dt.day
df_train_combined['month_of_year'] = df_train_combined['date'].dt.month
df_train_combined['day_of_week'] = df_train_combined['date'].dt.dayofweek

In [27]:
df_train_combined.drop(columns='date',inplace=True) 

In [28]:
df_train_combined.head(5)

,item_id,store_id,revenue,is_event,day_of_month,month_of_year,day_of_week
0,HOBBIES_1_001,CA_1,0.0,0,29,1,5
1,HOBBIES_1_002,CA_1,0.0,0,29,1,5
2,HOBBIES_1_003,CA_1,0.0,0,29,1,5
3,HOBBIES_1_004,CA_1,0.0,0,29,1,5
4,HOBBIES_1_005,CA_1,0.0,0,29,1,5


In [30]:
df_cleaned = df_train_combined.copy()

## 3. Build Pipeline

Import Pipeline from sklearn.pipeline, StandardScaler, OrdinalEncoder and OneHotEncoder from sklearn.preprocessing and SGDClassifier from sklearn.linear_model

In [75]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,OrdinalEncoder,OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

Create a Pipeline called num_transformer with one step that contains StandardScaler

In [32]:
num_transformer = Pipeline(
    steps=[
        ('scaler', StandardScaler())
    ]
)

Create a Pipeline called cat_transformer with one step that contains OneHotEncoder

In [33]:
cat_transformer = Pipeline(
    steps=[
        ('one_hot_encoder', OneHotEncoder(sparse=False, drop='first'))
    ]
)

Create a Pipeline called ord_transformer with one step that contains OrdinalEncoder

In [34]:
ord_transformer = Pipeline(
    steps=[
        ('ordinal_encoder', OrdinalEncoder())
    ]
)

Create a list called num_cols that will contain the list of columns that are numeric type

In [154]:
num_cols = ['is_event', 'day_of_month', 'month_of_year', 'day_of_week']

Import ColumnTransformer from sklearn.compose

In [36]:
from sklearn.compose import ColumnTransformer

Create a ColumnTransformer called preprocessor

In [155]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num_cols', num_transformer, num_cols),
        ('cat', cat_transformer, ['store_id']),
        ('ord', ord_transformer, ['item_id'])
    ]
)

## 4. Data Splitting

In [131]:
%reload_ext autoreload
import sys
sys.path.insert(0, '../..')
from src.data.sets import split_df_items,split_save
from sklearn.model_selection import train_test_split

Split the big dataset into smaller based on item department

In [52]:
HOBBIES_1_df=split_df_items(df_cleaned,'HOBBIES_1')
HOBBIES_2_df=split_df_items(df_cleaned,'HOBBIES_2')
FOODS_1_df=split_df_items(df_cleaned,'FOODS_1')
FOODS_2_df=split_df_items(df_cleaned,'FOODS_2')
FOODS_3_df=split_df_items(df_cleaned,'FOODS_3')
HOUSEHOLD_1_df=split_df_items(df_cleaned,'HOUSEHOLD_1')
HOUSEHOLD_2_df=split_df_items(df_cleaned,'HOUSEHOLD_2')


In [ ]:
df_cleaned["item_id"].count()

47107050

In [ ]:
HOBBIES_1_df["item_id"].count()+HOBBIES_2_df["item_id"].count()+FOODS_1_df["item_id"].count()+FOODS_2_df["item_id"].count()+FOODS_3_df["item_id"].count()+HOUSEHOLD_1_df["item_id"].count()+HOUSEHOLD_2_df["item_id"].count()

47107050

Split each department dataset into trainning and validating datasets and then save them to data/processs

In [132]:
split_save(HOBBIES_1_df,'HOBBIES_1_df','revenue')
split_save(HOBBIES_2_df,'HOBBIES_2_df','revenue')
split_save(FOODS_1_df,'FOODS_1_df','revenue')
split_save(FOODS_2_df,'FOODS_2_df','revenue')
split_save(FOODS_3_df,'FOODS_3_df','revenue')
split_save(HOUSEHOLD_1_df,'HOUSEHOLD_1_df','revenue')
split_save(HOUSEHOLD_2_df,'HOUSEHOLD_2_df','revenue')

## 5. Linear regression model

In [270]:
import warnings
warnings.filterwarnings('ignore')

In [259]:
from sklearn.linear_model import LinearRegression
import numpy as np

lr_pipe = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('lr', LinearRegression())
    ]
)

In [260]:
from src.data.sets import load_sets_train
from src.data.sets import load_sets_val
from joblib import dump
dep_list=['HOBBIES_1_df','HOBBIES_2_df','FOODS_1_df','FOODS_2_df','FOODS_3_df','HOUSEHOLD_1_df','HOUSEHOLD_2_df']

In [271]:
from sklearn.metrics import mean_squared_error
rmse_training=[]
rmse_validating=[]

for dep in dep_list:
    #Loading train and validate features and target
    X_train_temp,y_train_temp=load_sets_train(dep)
    X_val_temp,y_val_temp=load_sets_val(dep)
    #Fit the model by using pipline
    lr_pipe.fit(X_train_temp,y_train_temp)
    #Predict and calculate RMSE and MAE scores
    y_train_preds=lr_pipe.predict(X_train_temp)
    y_val_preds=lr_pipe.predict(X_val_temp)
    #Append to rmse lists
    rmse_training.append(np.sqrt(mean_squared_error(y_train_temp, y_train_preds)))
    rmse_validating.append(np.sqrt(mean_squared_error(y_val_temp, y_val_preds)))

In [272]:
scores = pd.DataFrame({'Department': dep_list, 'RMSE_training': rmse_training,'RMSE_validating': rmse_validating})
scores

,Department,RMSE_training,RMSE_validating
0,HOBBIES_1_df,7.743690,7.576550
1,HOBBIES_2_df,1.604205,1.627697
2,FOODS_1_df,7.651594,7.539368
3,FOODS_2_df,8.516221,8.462612
4,FOODS_3_df,12.540242,12.671741
5,HOUSEHOLD_1_df,9.054283,9.059017
6,HOUSEHOLD_2_df,4.009248,3.947123


## 6. Decision tree with tunning hyperparameters

In [275]:
dt_pipe = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('dt', DecisionTreeRegressor(max_depth=10,random_state=42))
    ]
)

In [276]:
from sklearn.metrics import mean_squared_error
rmse_training=[]
rmse_validating=[]

for dep in dep_list:
    #Loading train and validate features and target
    X_train_temp,y_train_temp=load_sets_train(dep)
    X_val_temp,y_val_temp=load_sets_val(dep)
    #Fit the model by using pipline
    dt_pipe.fit(X_train_temp,y_train_temp)
    #Predict and calculate RMSE and MAE scores
    y_train_preds=dt_pipe.predict(X_train_temp)
    y_val_preds=dt_pipe.predict(X_val_temp)
    #Append to rmse lists
    rmse_training.append(np.sqrt(mean_squared_error(y_train_temp, y_train_preds)))
    rmse_validating.append(np.sqrt(mean_squared_error(y_val_temp, y_val_preds)))

In [277]:
scores = pd.DataFrame({'Department': dep_list, 'RMSE_training': rmse_training,'RMSE_validating': rmse_validating})
scores

,Department,RMSE_training,RMSE_validating
0,HOBBIES_1_df,7.542530,7.385578
1,HOBBIES_2_df,1.555306,1.586430
2,FOODS_1_df,6.744371,6.719423
3,FOODS_2_df,7.849706,7.807749
4,FOODS_3_df,11.589590,11.722602
5,HOUSEHOLD_1_df,8.490327,8.518874
6,HOUSEHOLD_2_df,3.913705,3.859710


## 7. XGboost

In [ ]:
!pip install xgboost

In [278]:
from xgboost import XGBRegressor

xg_pipe = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('xg', XGBRegressor(random_state=42))
    ]
)

In [279]:
from sklearn.metrics import mean_squared_error
rmse_training=[]
rmse_validating=[]

for dep in dep_list:
    #Loading train and validate features and target
    X_train_temp,y_train_temp=load_sets_train(dep)
    X_val_temp,y_val_temp=load_sets_val(dep)
    #Fit the model by using pipline
    xg_pipe.fit(X_train_temp,y_train_temp)
    #Save models
    dump(xg_pipe,f'../../models/predictive/{dep}_xg_pipeline.joblib')
    #Predict and calculate RMSE and MAE scores
    y_train_preds=xg_pipe.predict(X_train_temp)
    y_val_preds=xg_pipe.predict(X_val_temp)
    #Append to rmse lists
    rmse_training.append(np.sqrt(mean_squared_error(y_train_temp, y_train_preds)))
    rmse_validating.append(np.sqrt(mean_squared_error(y_val_temp, y_val_preds)))

In [280]:
scores = pd.DataFrame({'Department': dep_list, 'RMSE_training': rmse_training,'RMSE_validating': rmse_validating})
scores

,Department,RMSE_training,RMSE_validating
0,HOBBIES_1_df,7.229247,7.099764
1,HOBBIES_2_df,1.515863,1.544727
2,FOODS_1_df,6.312459,6.242123
3,FOODS_2_df,7.417874,7.372960
4,FOODS_3_df,11.505372,11.634376
5,HOUSEHOLD_1_df,8.343368,8.358284
6,HOUSEHOLD_2_df,3.845009,3.787191
